In [1]:
import wandb
import os
import librosa

In [2]:
wandb.init(project="music-classification")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: isitarex. Use `wandb login --relogin` to force relogin


## 1. Create Artifact and Data processing

In [7]:
# read in the data on the data folder and store the names of the files, the labels and length of the files in a wandb table
def task_preprocess_data():
    # get the names of the files in the data folder
    
    # create a wandb table
    table = wandb.Table(columns=["file_name", "song","label", "length"])
    # loop through the files and get the label and length of each file
    for label in ['Like', 'Dislike']:
        files = os.listdir('data'+os.sep+label)
        for file in files:
            length = librosa.get_duration(filename='data/'+label+'/'+file)
            # load the file
            song = wandb.Audio('data/'+label+'/'+file)
            # add the file name, label and length to the table
            table.add_data(file, song, label, length)
    # log the table to wandb
    wandb.log({"data": table})

In [8]:
# run the task
task_preprocess_data()


In [6]:
# create an artifact for the data
data_artifact = wandb.Artifact('data', type='dataset')
# add the mp3 files to the artifact and the labels
data_artifact.add_dir('data_cut/Like', name='data/Like')
data_artifact.add_dir('data_cut/Dislike', name='data/Dislike')


# log the artifact to wandb
wandb.log_artifact(data_artifact)


wandb: Adding directory to artifact (./data_cut/Like)... Done. 0.5s
wandb: Adding directory to artifact (./data_cut/Dislike)... Done. 0.6s


In [9]:
wandb.finish()

## 2. Use artifact:

In [7]:
import os, warnings
import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

warnings.filterwarnings('ignore')
     

In [8]:
run = wandb.init(project='music-classification', job_type="data_split")

In [11]:
raw_data_at = run.use_artifact('data:latest', type='dataset')
path = Path(raw_data_at.download())

wandb: Downloading large artifact data:latest, 2053.61MB. 200 files... 
wandb:   200 of 200 files downloaded.  
Done. 0:0:1.4


In [12]:
path.ls()

(#1) [Path('artifacts/data:v0/data')]